In [1]:
import os
import h5py

In [2]:
import librosa

In [3]:
import itertools
from copy import copy
import numpy as np
import matplotlib.pyplot as plt
from collections import OrderedDict

In [4]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split

In [5]:
import numpy as np
import pandas as pd

In [6]:
from scipy.stats import kurtosis
from scipy.stats import skew

In [7]:
from collections import defaultdict
import pickle

In [8]:
import ast

In [9]:
#I took help from github link: https://github.com/celestinhermez/music-genre-classification/blob/master/data_preprocessing/create_audio_features.ipynb
# path to the small directory
SMALL_AUDIO_DIR = 'E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small'

# function to get the paths to all the songs in the small dataset
def audio_paths(AUDIO_DIR):
    AUDIO_PATHS = []
    # iterate through all the directories with songs in them
    for path in [os.path.join('E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small', p) 
                 for p in os.listdir('E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small') 
                 if not (p.endswith('checksums') or p.endswith('.txt') or p.endswith('.DS_Store'))]:
        # add all songs to the list
        AUDIO_PATHS = AUDIO_PATHS + [os.path.join(path, track).replace('\\', '/') for track in os.listdir(path)]
    
    return AUDIO_PATHS

# store all the small paths
SMALL_PATHS = audio_paths(SMALL_AUDIO_DIR)

In [10]:
print(SMALL_PATHS[0:15])

['E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/000/000002.mp3', 'E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/000/000005.mp3', 'E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/000/000010.mp3', 'E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/000/000140.mp3', 'E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/000/000141.mp3', 'E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/000/000148.mp3', 'E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/000/000182.mp3', 'E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/000/000190.mp3', 'E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/000/000193.mp3', 'E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/000/000194.mp3', 'E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/000/000197.mp3', 'E:/8th Semester/CSE4836-Patter

In [11]:
t=pd.read_csv('E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_metadata/tracks.csv')
t.head(3)

C:\Users\HP\Anaconda\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (0,1,5,6,8,12,18,20,21,22,24,33,34,38,39,44,47,49) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,Unnamed: 0,album,album.1,album.2,album.3,album.4,album.5,album.6,album.7,album.8,...,track.10,track.11,track.12,track.13,track.14,track.15,track.16,track.17,track.18,track.19
0,NaN,comments,date_created,date_released,engineer,favorites,id,information,listens,producer,...,information,interest,language_code,license,listens,lyricist,number,publisher,tags,title
1,track_id,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,0,2008-11-26 01:44:45,2009-01-05 00:00:00,NaN,4,1,<p></p>,6073,NaN,...,NaN,4656,en,Attribution-NonCommercial-ShareAlike 3.0 Inter...,1293,NaN,3,NaN,[],Food


In [12]:
# function to load metadata
# adapted from https://github.com/mdeff/fma/blob/master/utils.py
def metadata_load(filepath):

    filename = os.path.basename(filepath)

    if 'features' in filename:
        return pd.read_csv(filepath, index_col=0, header=[0, 1, 2])

    if 'echonest' in filename:
        return pd.read_csv(filepath, index_col=0, header=[0, 1, 2])

    if 'genres' in filename:
        return pd.read_csv(filepath, index_col=0)

    if 'tracks' in filename:
        tracks = pd.read_csv(filepath, index_col=0, header=[0, 1])

        COLUMNS = [('track', 'tags'), ('album', 'tags'), ('artist', 'tags'),
                   ('track', 'genres'), ('track', 'genres_all')]
        for column in COLUMNS:
            tracks[column] = tracks[column].map(ast.literal_eval)

        COLUMNS = [('track', 'date_created'), ('track', 'date_recorded'),
                   ('album', 'date_created'), ('album', 'date_released'),
                   ('artist', 'date_created'), ('artist', 'active_year_begin'),
                   ('artist', 'active_year_end')]
        for column in COLUMNS:
            tracks[column] = pd.to_datetime(tracks[column])

        SUBSETS = ('small', 'medium', 'large')
        try:
            tracks['set', 'subset'] = tracks['set', 'subset'].astype(
                    pd.CategoricalDtype(categories=SUBSETS, ordered=True))
        except ValueError:
            # the categories and ordered arguments were removed in pandas 0.25
            tracks['set', 'subset'] = tracks['set', 'subset'].astype(
                     pd.CategoricalDtype(categories=SUBSETS, ordered=True))

        COLUMNS = [('track', 'genre_top'), ('track', 'license'),
                   ('album', 'type'), ('album', 'information'),
                   ('artist', 'bio')]
        for column in COLUMNS:
            tracks[column] = tracks[column].astype('category')

        return tracks

In [13]:
# function to get genre information for each track ID
def track_genre_information(GENRE_PATH, TRACKS_PATH, FILE_PATHS, subset):
    """
    GENRE_PATH (str): path to the csv with the genre metadata
    TRACKS_PATH (str): path to the csv with the track metadata
    FILE_PATHS (list): list of paths to the mp3 files
    subset (str): the subset of the data desired
    """
    # get the genre information
    genres = pd.read_csv(GENRE_PATH)

    # load metadata on all the tracks
    tracks = metadata_load(TRACKS_PATH)
    print(tracks.head(5))

    # focus on the specific subset tracks
    subset_tracks = tracks[tracks['set', 'subset'] <= subset]

    # extract track ID and genre information for each track
    subset_tracks_genre = np.array([np.array(subset_tracks.index), 
                                  np.array(subset_tracks['track', 'genre_top'])]).T
    
    # extract track indices from the file paths
    track_indices = []
    for path in FILE_PATHS:
        track_indices.append(path.split('/')[-1].split('.')[0].lstrip('0'))

    # get the genre associated with each file path, thanks to the path ID
    print(track_indices)
    track_indices = pd.DataFrame({'file_path':FILE_PATHS,'track_id':np.array(track_indices).astype(int)})
    tracks_genre_df = pd.DataFrame({'track_id': subset_tracks_genre[:,0], 'genre': subset_tracks_genre[:,1]})
    track_genre_data = track_indices.merge(tracks_genre_df, how='left')
    
    # label classes with numbers
    encoder = LabelEncoder()
    track_genre_data['genre_nb'] = encoder.fit_transform(track_genre_data.genre)
    
    return track_genre_data

# get genre information for all tracks from the small subset
GENRE_PATH = 'E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_metadata/genres.csv'
TRACKS_PATH = 'E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_metadata/tracks.csv'
subset = 'small'

small_tracks_genre = track_genre_information(GENRE_PATH, TRACKS_PATH, SMALL_PATHS, subset)

            album                                                          \
         comments        date_created date_released engineer favorites id   
track_id                                                                    
2               0 2008-11-26 01:44:45    2009-01-05      NaN         4  1   
3               0 2008-11-26 01:44:45    2009-01-05      NaN         4  1   
5               0 2008-11-26 01:44:45    2009-01-05      NaN         4  1   
10              0 2008-11-26 01:45:08    2008-02-06      NaN         4  6   
20              0 2008-11-26 01:45:05    2009-01-06      NaN         2  4   

                                                                           \
                                        information listens producer tags   
track_id                                                                    
2                                           <p></p>    6073      NaN   []   
3                                           <p></p>    6073      NaN   []  

In [14]:
small_tracks_genre.head(4)

,file_path,track_id,genre,genre_nb
0,E:/8th Semester/CSE4836-Pattern Recognition La...,2,Hip-Hop,3
1,E:/8th Semester/CSE4836-Pattern Recognition La...,5,Hip-Hop,3
2,E:/8th Semester/CSE4836-Pattern Recognition La...,10,Pop,6
3,E:/8th Semester/CSE4836-Pattern Recognition La...,140,Folk,2


In [15]:
import scipy.io.wavfile as wav
import numpy as np
from tempfile import TemporaryFile
import os
import pickle
import random 
import operator
import math
import numpy as np
import os
import librosa
import itertools
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import kurtosis
from scipy.stats import skew

In [25]:
def create_small_sample(X, y, window = 0.25, overlap = 0.5):
  temp_X = []
  temp_y = []

  row = X.shape[0]
  sample_on_window = int(row*window)
  iter = int(sample_on_window*(1.-overlap))
    
  last_iter = row - sample_on_window + iter

  splitted_song = [X[i:i+sample_on_window] for i in range(0, last_iter , iter)]
  for s in splitted_song:
      if s.shape[0] == sample_on_window:
        temp_X.append(s)
        temp_y.append(y)

  return np.array(temp_X), np.array(temp_y)

In [17]:
def to_mel(song, n_fft=1024, hop_length=256):
  return librosa.feature.melspectrogram(song, n_fft=n_fft,hop_length=hop_length, n_mels=128)[:,:,np.newaxis]

In [18]:
def get_melspectrogram(song, n_fft=1024, hop_length=256):
  mels_by_split = map(to_mel, song)
  return np.array(list(mels_by_split))

In [19]:
def split_then_mels(X, y):
  arr_splits = []
  arr_genres = []
  f = 0
  for feat, genre in zip(X, y):
    print("Reading file: {}".format(feat))
    print(f)
    f = f+1
    signal, rate = librosa.load(feat)
    signal = signal[:take_samples]

    # Splitsongs into 0.15 of each to take sample of 1.5 seconds only
    signals, y = create_small_sample(signal, genre)

    # Get Labrosa Melspectogram
    splits = get_melspectrogram(signals)

        
    arr_genres.extend(y)
    arr_splits.extend(splits)
    
  #ret_files = np.array(arr_splits)
  #y_val = np.array(arr_genres)
  return arr_splits, arr_genres

In [20]:
take_samples = 660000
#Creating a Dictionary of Genres
genres = {
    'Hip-Hop': 0, 'Pop': 1, 'Folk': 2, 'Experimental': 3, 'Rock': 4, 
    'International': 5, 'Electronic': 6, 'Instrumental': 7
}

In [21]:
list_file = []
list_genres = []

for ind in small_tracks_genre.index:
  song_link = small_tracks_genre['file_path'][ind]
  song_genre = small_tracks_genre['genre'][ind]
  list_file.append(song_link)
  list_genres.append(genres[song_genre])

X_train, X_test, y_train, y_test = train_test_split(list_file, list_genres, test_size=0.25, random_state=47, stratify=list_genres)

In [22]:
print(len(X_train) , len(y_train), len(X_test), len(y_test))

5997 5997 2000 2000


In [23]:
X_train1 = X_train[0:1000]
y_train1 = y_train[0:1000]
X_train2 = X_train[1000:2000]
y_train2 = y_train[1000:2000]
X_train3 = X_train[2000:3000]
y_train3 = y_train[2000:3000]
X_train4 = X_train[3000:4000]
y_train4 = y_train[3000:4000]
X_train5 = X_train[4000:5000]
y_train5 = y_train[4000:5000]
X_train6 = X_train[5000:]
y_train6 = y_train[5000:]
X_test1 = X_test[0:1000]
y_test1 = y_test[0:1000]
X_test2 = X_test[1000:]
y_test2 = y_test[1000:]
print(len(X_train6), len(y_train6))

997 997


# Train-1

In [40]:
X_train12, y_train12 = split_then_mels(X_train2, y_train2)
a = X_train12
b = y_train12

Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/081/081194.mp3
0
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/071/071255.mp3
1
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/052/052642.mp3
2
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/135/135226.mp3
3
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/111/111228.mp3
4
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/120/120200.mp3
5
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/108/108866.mp3
6
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/089/089857.mp3
7
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/090/090695.mp3
8
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/125/125

Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/054/054570.mp3
81
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/117/117666.mp3
82
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/064/064991.mp3
83
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/068/068837.mp3
84
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/006/006342.mp3
85
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/102/102114.mp3
86
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/061/061668.mp3
87
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/054/054666.mp3
88
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/038/038323.mp3
89
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_smal

Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/003/003895.mp3
161
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/117/117631.mp3
162
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/033/033070.mp3
163
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/138/138063.mp3
164
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/145/145752.mp3
165
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/097/097374.mp3
166
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/132/132961.mp3
167
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/134/134054.mp3
168
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/017/017462.mp3
169
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset

Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/115/115930.mp3
241
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/042/042751.mp3
242
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/094/094803.mp3
243
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/093/093943.mp3
244
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/064/064856.mp3
245
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/143/143305.mp3
246
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/075/075754.mp3
247
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/004/004509.mp3
248
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/057/057179.mp3
249
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset

Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/006/006783.mp3
321
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/095/095908.mp3
322
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/055/055811.mp3
323
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/087/087430.mp3
324
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/105/105196.mp3
325
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/038/038825.mp3
326
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/098/098550.mp3
327
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/122/122649.mp3
328
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/114/114372.mp3
329
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset

...

Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/096/096718.mp3
641
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/129/129918.mp3
642
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/116/116389.mp3
643
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/038/038911.mp3
644
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/145/145074.mp3
645
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/139/139934.mp3
646
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/089/089445.mp3
647
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/126/126584.mp3
648
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/060/060146.mp3
649
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset

Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/126/126428.mp3
721
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/053/053937.mp3
722
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/125/125286.mp3
723
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/067/067367.mp3
724
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/116/116456.mp3
725
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/075/075418.mp3
726
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/069/069761.mp3
727
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/015/015625.mp3
728
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/123/123440.mp3
729
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset

Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/036/036380.mp3
801
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/088/088848.mp3
802
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/082/082917.mp3
803
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/081/081946.mp3
804
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/133/133780.mp3
805
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/000/000140.mp3
806
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/126/126219.mp3
807
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/047/047659.mp3
808
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/064/064542.mp3
809
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset

Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/129/129694.mp3
881
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/075/075230.mp3
882
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/106/106457.mp3
883
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/132/132122.mp3
884
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/133/133975.mp3
885
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/099/099368.mp3
886
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/141/141304.mp3
887
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/147/147815.mp3
888
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/123/123867.mp3
889
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset

Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/119/119895.mp3
961
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/054/054735.mp3
962
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/012/012986.mp3
963
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/098/098681.mp3
964
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/056/056517.mp3
965
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/118/118058.mp3
966
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/063/063149.mp3
967
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/119/119571.mp3
968
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/074/074706.mp3
969
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset

In [41]:
w = len(a)

In [42]:
now_dunno = np.array(a)

In [43]:
np.save('E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/Dataset/ForCNNList/train_setX1', now_dunno) 

In [29]:
st = []

for i in range(w):
    for j in range(len(a[i])):
        if len(a[i][j]) != 645:
            st.append(i)

In [30]:
print(len(st))

896


In [31]:
print(len(a))

7000


In [32]:
print(st)

[861, 861, 861, 861, 861, 861, 861, 861, 861, 861, 861, 861, 861, 861, 861, 861, 861, 861, 861, 861, 861, 861, 861, 861, 861, 861, 861, 861, 861, 861, 861, 861, 861, 861, 861, 861, 861, 861, 861, 861, 861, 861, 861, 861, 861, 861, 861, 861, 861, 861, 861, 861, 861, 861, 861, 861, 861, 861, 861, 861, 861, 861, 861, 861, 861, 861, 861, 861, 861, 861, 861, 861, 861, 861, 861, 861, 861, 861, 861, 861, 861, 861, 861, 861, 861, 861, 861, 861, 861, 861, 861, 861, 861, 861, 861, 861, 861, 861, 861, 861, 861, 861, 861, 861, 861, 861, 861, 861, 861, 861, 861, 861, 861, 861, 861, 861, 861, 861, 861, 861, 861, 861, 861, 861, 861, 861, 861, 861, 862, 862, 862, 862, 862, 862, 862, 862, 862, 862, 862, 862, 862, 862, 862, 862, 862, 862, 862, 862, 862, 862, 862, 862, 862, 862, 862, 862, 862, 862, 862, 862, 862, 862, 862, 862, 862, 862, 862, 862, 862, 862, 862, 862, 862, 862, 862, 862, 862, 862, 862, 862, 862, 862, 862, 862, 862, 862, 862, 862, 862, 862, 862, 862, 862, 862, 862, 862, 862, 862, 862, 862,

In [33]:
print(len(a[2341][0]))

645


In [34]:
st = list(set(st))
print(len(st))

7


In [35]:
print(st)

[864, 865, 866, 867, 861, 862, 863]


In [36]:
print(len(a),len(b))

7000 7000


In [37]:
del_len = len(st)
for j in range(del_len):
    ind=del_len-j-1
    a.pop(st[ind])
    b.pop(st[ind])

In [38]:
print(len(a),len(b))

6993 6993


In [39]:
now_dunno = np.array(a)

<ipython-input-39-e8af67fa0fe2>:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  now_dunno = np.array(a)


ValueError: could not broadcast input array from shape (128,645,1) into shape (128)

In [59]:
import pickle
fileXtrain = 'E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/Dataset/ForCNNList/train_setX1'
outfile = open(fileXtrain,'wb')
pickle.dump(a,outfile)
outfile.close()

In [61]:
fileXtrain = 'E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/Dataset/ForCNNList/train_sety1'
outfile = open(fileXtrain,'wb')
pickle.dump(b,outfile)
outfile.close()

# Train - 2

In [62]:
X_train12, y_train12 = split_then_mels(X_train2, y_train2)
a = X_train12
b = y_train12

Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/081/081194.mp3
0
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/071/071255.mp3
1
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/052/052642.mp3
2
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/135/135226.mp3
3
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/111/111228.mp3
4
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/120/120200.mp3
5
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/108/108866.mp3
6
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/089/089857.mp3
7
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/090/090695.mp3
8
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/125/125

Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/054/054570.mp3
81
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/117/117666.mp3
82
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/064/064991.mp3
83
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/068/068837.mp3
84
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/006/006342.mp3
85
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/102/102114.mp3
86
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/061/061668.mp3
87
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/054/054666.mp3
88
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/038/038323.mp3
89
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_smal

Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/003/003895.mp3
161
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/117/117631.mp3
162
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/033/033070.mp3
163
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/138/138063.mp3
164
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/145/145752.mp3
165
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/097/097374.mp3
166
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/132/132961.mp3
167
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/134/134054.mp3
168
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/017/017462.mp3
169
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset

Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/115/115930.mp3
241
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/042/042751.mp3
242
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/094/094803.mp3
243
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/093/093943.mp3
244
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/064/064856.mp3
245
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/143/143305.mp3
246
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/075/075754.mp3
247
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/004/004509.mp3
248
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/057/057179.mp3
249
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset

Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/006/006783.mp3
321
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/095/095908.mp3
322
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/055/055811.mp3
323
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/087/087430.mp3
324
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/105/105196.mp3
325
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/038/038825.mp3
326
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/098/098550.mp3
327
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/122/122649.mp3
328
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/114/114372.mp3
329
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset

...

Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/096/096718.mp3
641
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/129/129918.mp3
642
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/116/116389.mp3
643
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/038/038911.mp3
644
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/145/145074.mp3
645
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/139/139934.mp3
646
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/089/089445.mp3
647
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/126/126584.mp3
648
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/060/060146.mp3
649
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset

Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/126/126428.mp3
721
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/053/053937.mp3
722
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/125/125286.mp3
723
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/067/067367.mp3
724
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/116/116456.mp3
725
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/075/075418.mp3
726
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/069/069761.mp3
727
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/015/015625.mp3
728
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/123/123440.mp3
729
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset

Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/036/036380.mp3
801
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/088/088848.mp3
802
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/082/082917.mp3
803
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/081/081946.mp3
804
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/133/133780.mp3
805
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/000/000140.mp3
806
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/126/126219.mp3
807
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/047/047659.mp3
808
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/064/064542.mp3
809
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset

Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/129/129694.mp3
881
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/075/075230.mp3
882
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/106/106457.mp3
883
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/132/132122.mp3
884
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/133/133975.mp3
885
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/099/099368.mp3
886
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/141/141304.mp3
887
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/147/147815.mp3
888
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/123/123867.mp3
889
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset

Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/119/119895.mp3
961
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/054/054735.mp3
962
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/012/012986.mp3
963
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/098/098681.mp3
964
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/056/056517.mp3
965
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/118/118058.mp3
966
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/063/063149.mp3
967
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/119/119571.mp3
968
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/074/074706.mp3
969
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset

In [63]:
w = len(a)

In [64]:
st = []

for i in range(w):
    for j in range(len(a[i])):
        if len(a[i][j]) != 258:
            st.append(i)

In [65]:
st = list(set(st))
print(len(st))

0


In [66]:
del_len = len(st)
for j in range(del_len):
    ind=del_len-j-1
    a.pop(st[ind])
    b.pop(st[ind])

In [67]:
print(len(a),len(b))

19000 19000


In [68]:
fileXtrain = 'E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/Dataset/ForCNNList/train_setX2'
outfile = open(fileXtrain,'wb')
pickle.dump(a,outfile)
outfile.close()

In [69]:
fileXtrain = 'E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/Dataset/ForCNNList/train_sety2'
outfile = open(fileXtrain,'wb')
pickle.dump(b,outfile)
outfile.close()

# Train - 3

In [70]:
X_train12, y_train12 = split_then_mels(X_train3, y_train3)
a = X_train12
b = y_train12

Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/121/121665.mp3
0
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/045/045153.mp3
1
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/114/114223.mp3
2
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/143/143220.mp3
3
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/116/116453.mp3
4
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/095/095722.mp3
5
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/113/113334.mp3
6
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/126/126183.mp3
7
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/050/050446.mp3
8
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/126/126

Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/033/033422.mp3
81
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/097/097958.mp3
82
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/062/062193.mp3
83
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/090/090625.mp3
84
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/113/113022.mp3
85
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/116/116451.mp3
86
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/013/013539.mp3
87
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/043/043857.mp3
88
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/118/118672.mp3
89
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_smal

Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/098/098620.mp3
161
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/027/027613.mp3
162
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/122/122645.mp3
163
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/058/058541.mp3
164
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/043/043599.mp3
165
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/107/107128.mp3
166
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/074/074388.mp3
167
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/052/052036.mp3
168
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/080/080775.mp3
169
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset

Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/012/012654.mp3
241
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/087/087159.mp3
242
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/075/075396.mp3
243
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/117/117627.mp3
244
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/131/131933.mp3
245
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/088/088861.mp3
246
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/091/091933.mp3
247
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/038/038910.mp3
248
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/026/026012.mp3
249
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset

Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/072/072206.mp3
321
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/067/067308.mp3
322
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/048/048491.mp3
323
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/123/123474.mp3
324
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/085/085837.mp3
325
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/087/087192.mp3
326
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/126/126102.mp3
327
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/026/026174.mp3
328
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/057/057943.mp3
329
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset

...

Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/064/064567.mp3
641
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/025/025124.mp3
642
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/052/052862.mp3
643
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/148/148186.mp3
644
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/019/019179.mp3
645
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/137/137900.mp3
646
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/086/086799.mp3
647
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/111/111399.mp3
648
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/108/108461.mp3
649
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset

Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/070/070005.mp3
721
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/149/149078.mp3
722
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/145/145020.mp3
723
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/014/014315.mp3
724
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/116/116098.mp3
725
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/032/032756.mp3
726
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/130/130710.mp3
727
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/080/080000.mp3
728
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/033/033538.mp3
729
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset

Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/043/043028.mp3
801
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/053/053726.mp3
802
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/129/129046.mp3
803
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/116/116873.mp3
804
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/061/061884.mp3
805
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/110/110744.mp3
806
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/105/105824.mp3
807
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/072/072130.mp3
808
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/004/004100.mp3
809
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset

Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/129/129055.mp3
881
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/053/053727.mp3
882
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/061/061592.mp3
883
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/021/021231.mp3
884
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/019/019438.mp3
885
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/054/054625.mp3
886
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/011/011766.mp3
887
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/142/142081.mp3
888
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/052/052947.mp3
889
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset

Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/122/122623.mp3
961
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/127/127211.mp3
962
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/021/021774.mp3
963
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/092/092955.mp3
964
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/150/150017.mp3
965
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/127/127062.mp3
966
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/114/114534.mp3
967
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/117/117613.mp3
968
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/082/082931.mp3
969
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset

In [71]:
w = len(a)

In [72]:
st = []

for i in range(w):
    for j in range(len(a[i])):
        if len(a[i][j]) != 258:
            st.append(i)

In [73]:
st = list(set(st))
print(len(st))

0


In [74]:
del_len = len(st)
for j in range(del_len):
    ind=del_len-j-1
    a.pop(st[ind])
    b.pop(st[ind])

In [75]:
print(len(a),len(b))

19000 19000


In [76]:
fileXtrain = 'E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/Dataset/ForCNNList/train_setX3'
outfile = open(fileXtrain,'wb')
pickle.dump(a,outfile)
outfile.close()

In [77]:
fileXtrain = 'E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/Dataset/ForCNNList/train_sety3'
outfile = open(fileXtrain,'wb')
pickle.dump(b,outfile)
outfile.close()

# Train - 4

In [78]:
X_train12, y_train12 = split_then_mels(X_train4, y_train4)
a = X_train12
b = y_train12

Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/101/101111.mp3
0
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/115/115273.mp3
1
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/069/069826.mp3
2
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/052/052448.mp3
3
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/043/043596.mp3
4
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/125/125001.mp3
5
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/001/001720.mp3
6
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/018/018146.mp3
7
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/062/062655.mp3
8
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/148/148

Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/122/122622.mp3
81
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/035/035539.mp3
82
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/108/108428.mp3
83
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/010/010521.mp3
84
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/124/124915.mp3
85
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/021/021403.mp3
86
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/110/110107.mp3
87
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/069/069768.mp3
88
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/122/122358.mp3
89
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_smal

Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/146/146716.mp3
161
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/016/016821.mp3
162
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/016/016745.mp3
163
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/141/141291.mp3
164
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/088/088486.mp3
165
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/068/068897.mp3
166
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/070/070424.mp3
167
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/087/087189.mp3
168
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/139/139776.mp3
169
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset

Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/063/063874.mp3
241
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/116/116407.mp3
242
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/067/067011.mp3
243
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/143/143300.mp3
244
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/129/129997.mp3
245
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/015/015488.mp3
246
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/036/036258.mp3
247
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/100/100976.mp3
248
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/120/120297.mp3
249
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset

Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/003/003779.mp3
321
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/000/000615.mp3
322
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/060/060476.mp3
323
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/097/097697.mp3
324
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/087/087100.mp3
325
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/058/058117.mp3
326
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/046/046732.mp3
327
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/116/116235.mp3
328
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/105/105141.mp3
329
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset

...

Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/025/025601.mp3
641
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/054/054433.mp3
642
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/064/064332.mp3
643
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/109/109543.mp3
644
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/081/081650.mp3
645
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/057/057820.mp3
646
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/141/141282.mp3
647
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/000/000706.mp3
648
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/144/144470.mp3
649
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset

Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/120/120326.mp3
721
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/006/006605.mp3
722
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/011/011918.mp3
723
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/075/075607.mp3
724
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/055/055808.mp3
725
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/046/046842.mp3
726
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/073/073774.mp3
727
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/026/026021.mp3
728
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/080/080520.mp3
729
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset

Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/096/096696.mp3
801
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/098/098657.mp3
802
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/010/010693.mp3
803
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/094/094628.mp3
804
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/060/060753.mp3
805
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/016/016338.mp3
806
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/010/010671.mp3
807
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/142/142097.mp3
808
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/146/146726.mp3
809
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset

Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/125/125777.mp3
881
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/055/055832.mp3
882
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/017/017631.mp3
883
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/107/107186.mp3
884
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/089/089816.mp3
885
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/040/040241.mp3
886
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/123/123979.mp3
887
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/059/059677.mp3
888
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/123/123441.mp3
889
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset

Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/123/123509.mp3
961
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/001/001680.mp3
962
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/135/135028.mp3
963
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/148/148510.mp3
964
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/029/029528.mp3
965
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/127/127660.mp3
966
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/108/108019.mp3
967
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/131/131325.mp3
968
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/031/031889.mp3
969
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset

In [79]:
w = len(a)

In [80]:
st = []

for i in range(w):
    for j in range(len(a[i])):
        if len(a[i][j]) != 258:
            st.append(i)

In [81]:
st = list(set(st))
print(len(st))

0


In [82]:
del_len = len(st)
for j in range(del_len):
    ind=del_len-j-1
    a.pop(st[ind])
    b.pop(st[ind])

In [83]:
print(len(a),len(b))

19000 19000


In [84]:
fileXtrain = 'E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/Dataset/ForCNNList/train_setX4'
outfile = open(fileXtrain,'wb')
pickle.dump(a,outfile)
outfile.close()

In [85]:
fileXtrain = 'E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/Dataset/ForCNNList/train_sety4'
outfile = open(fileXtrain,'wb')
pickle.dump(b,outfile)
outfile.close()

# Train - 5

In [86]:
X_train12, y_train12 = split_then_mels(X_train5, y_train5)
a = X_train12
b = y_train12

Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/111/111390.mp3
0
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/087/087325.mp3
1
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/014/014583.mp3
2
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/054/054470.mp3
3
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/145/145887.mp3
4
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/010/010435.mp3
5
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/127/127208.mp3
6
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/056/056472.mp3
7
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/117/117472.mp3
8
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/142/142

Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/130/130963.mp3
81
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/121/121652.mp3
82
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/107/107852.mp3
83
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/044/044947.mp3
84
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/098/098567.mp3
85
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/120/120471.mp3
86
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/126/126283.mp3
87
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/107/107805.mp3
88
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/129/129372.mp3
89
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_smal

Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/113/113946.mp3
161
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/078/078850.mp3
162
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/107/107433.mp3
163
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/118/118507.mp3
164
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/092/092573.mp3
165
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/042/042126.mp3
166
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/014/014568.mp3
167
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/113/113949.mp3
168
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/142/142567.mp3
169
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset

Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/012/012174.mp3
241
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/134/134586.mp3
242
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/099/099374.mp3
243
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/144/144212.mp3
244
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/108/108302.mp3
245
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/141/141561.mp3
246
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/113/113335.mp3
247
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/108/108992.mp3
248
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/070/070655.mp3
249
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset

Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/125/125621.mp3
321
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/107/107584.mp3
322
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/067/067163.mp3
323
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/006/006778.mp3
324
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/045/045101.mp3
325
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/027/027798.mp3
326
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/122/122400.mp3
327
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/004/004519.mp3
328
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/012/012052.mp3
329
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset

...

Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/079/079087.mp3
641
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/052/052579.mp3
642
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/042/042135.mp3
643
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/126/126234.mp3
644
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/138/138040.mp3
645
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/128/128882.mp3
646
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/121/121813.mp3
647
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/099/099372.mp3
648
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/087/087121.mp3
649
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset

Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/148/148211.mp3
721
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/133/133433.mp3
722
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/060/060045.mp3
723
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/140/140871.mp3
724
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/012/012189.mp3
725
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/068/068536.mp3
726
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/137/137721.mp3
727
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/006/006357.mp3
728
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/063/063789.mp3
729
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset

Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/148/148031.mp3
801
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/031/031389.mp3
802
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/121/121570.mp3
803
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/064/064035.mp3
804
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/075/075436.mp3
805
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/055/055233.mp3
806
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/004/004688.mp3
807
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/051/051262.mp3
808
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/044/044802.mp3
809
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset

Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/042/042020.mp3
881
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/109/109497.mp3
882
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/064/064408.mp3
883
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/064/064988.mp3
884
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/067/067470.mp3
885
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/134/134792.mp3
886
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/137/137990.mp3
887
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/060/060510.mp3
888
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/038/038818.mp3
889
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset

Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/013/013578.mp3
961
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/107/107955.mp3
962
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/110/110111.mp3
963
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/071/071692.mp3
964
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/097/097540.mp3
965
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/086/086993.mp3
966
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/122/122647.mp3
967
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/122/122003.mp3
968
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/123/123468.mp3
969
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset

In [87]:
w = len(a)

In [88]:
st = []

for i in range(w):
    for j in range(len(a[i])):
        if len(a[i][j]) != 258:
            st.append(i)

In [89]:
st = list(set(st))
print(len(st))

19


In [90]:
del_len = len(st)
for j in range(del_len):
    ind=del_len-j-1
    a.pop(st[ind])
    b.pop(st[ind])

In [91]:
print(len(a),len(b))

18981 18981


In [92]:
fileXtrain = 'E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/Dataset/ForCNNList/train_setX5'
outfile = open(fileXtrain,'wb')
pickle.dump(a,outfile)
outfile.close()

In [93]:
fileXtrain = 'E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/Dataset/ForCNNList/train_sety5'
outfile = open(fileXtrain,'wb')
pickle.dump(b,outfile)
outfile.close()

# Train - 6

In [94]:
X_train12, y_train12 = split_then_mels(X_train6, y_train6)
a = X_train12
b = y_train12

Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/130/130691.mp3
0
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/121/121663.mp3
1
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/127/127402.mp3
2
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/104/104357.mp3
3
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/096/096694.mp3
4
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/024/024426.mp3
5
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/075/075782.mp3
6
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/115/115288.mp3
7
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/017/017634.mp3
8
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/010/010

Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/006/006390.mp3
81
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/126/126429.mp3
82
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/059/059676.mp3
83
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/064/064338.mp3
84
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/039/039658.mp3
85
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/000/000716.mp3
86
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/129/129883.mp3
87
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/029/029719.mp3
88
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/047/047665.mp3
89
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_smal

Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/132/132041.mp3
161
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/129/129392.mp3
162
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/030/030090.mp3
163
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/042/042016.mp3
164
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/121/121594.mp3
165
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/123/123965.mp3
166
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/086/086725.mp3
167
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/111/111188.mp3
168
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/071/071251.mp3
169
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset

Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/007/007492.mp3
241
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/132/132787.mp3
242
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/071/071253.mp3
243
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/043/043883.mp3
244
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/152/152261.mp3
245
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/087/087153.mp3
246
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/059/059654.mp3
247
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/118/118952.mp3
248
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/141/141375.mp3
249
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset

Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/091/091178.mp3
321
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/055/055709.mp3
322
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/148/148121.mp3
323
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/104/104278.mp3
324
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/095/095486.mp3
325
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/013/013199.mp3
326
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/073/073568.mp3
327
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/108/108339.mp3
328
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/052/052501.mp3
329
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset

...

Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/111/111335.mp3
641
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/004/004073.mp3
642
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/114/114373.mp3
643
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/095/095310.mp3
644
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/078/078038.mp3
645
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/087/087641.mp3
646
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/133/133332.mp3
647
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/121/121453.mp3
648
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/057/057442.mp3
649
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset

Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/116/116704.mp3
721
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/119/119063.mp3
722
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/069/069193.mp3
723
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/012/012352.mp3
724
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/053/053863.mp3
725
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/139/139769.mp3
726
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/000/000705.mp3
727
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/052/052389.mp3
728
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/137/137593.mp3
729
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset

Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/129/129095.mp3
801
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/108/108036.mp3
802
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/074/074382.mp3
803
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/115/115769.mp3
804
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/130/130016.mp3
805
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/123/123484.mp3
806
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/051/051203.mp3
807
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/056/056804.mp3
808
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/115/115267.mp3
809
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset

Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/001/001683.mp3
881
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/118/118196.mp3
882
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/108/108969.mp3
883
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/073/073773.mp3
884
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/095/095910.mp3
885
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/106/106566.mp3
886
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/014/014693.mp3
887
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/012/012049.mp3
888
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/047/047077.mp3
889
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset

Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/026/026016.mp3
961
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/019/019415.mp3
962
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/143/143059.mp3
963
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/020/020050.mp3
964
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/122/122634.mp3
965
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/000/000997.mp3
966
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/127/127273.mp3
967
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/050/050539.mp3
968
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/069/069789.mp3
969
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset

In [95]:
w = len(a)

In [96]:
st = []

for i in range(w):
    for j in range(len(a[i])):
        if len(a[i][j]) != 258:
            st.append(i)

In [97]:
st = list(set(st))
print(len(st))

19


In [98]:
del_len = len(st)
for j in range(del_len):
    ind=del_len-j-1
    a.pop(st[ind])
    b.pop(st[ind])

In [99]:
print(len(a),len(b))

18924 18924


In [100]:
fileXtrain = 'E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/Dataset/ForCNNList/train_setX6'
outfile = open(fileXtrain,'wb')
pickle.dump(a,outfile)
outfile.close()

In [101]:
fileXtrain = 'E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/Dataset/ForCNNList/train_sety6'
outfile = open(fileXtrain,'wb')
pickle.dump(b,outfile)
outfile.close()

# Test - 1

In [102]:
X_train12, y_train12 = split_then_mels(X_test1, y_test1)
a = X_train12
b = y_train12

Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/052/052522.mp3
0
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/145/145748.mp3
1
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/110/110086.mp3
2
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/092/092561.mp3
3
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/150/150062.mp3
4
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/064/064520.mp3
5
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/058/058215.mp3
6
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/136/136928.mp3
7
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/088/088873.mp3
8
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/036/036

Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/149/149370.mp3
81
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/006/006779.mp3
82
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/010/010674.mp3
83
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/038/038890.mp3
84
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/093/093921.mp3
85
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/026/026020.mp3
86
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/014/014734.mp3
87
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/105/105827.mp3
88
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/043/043030.mp3
89
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_smal

Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/097/097585.mp3
161
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/075/075314.mp3
162
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/005/005376.mp3
163
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/132/132781.mp3
164
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/104/104356.mp3
165
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/007/007490.mp3
166
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/056/056465.mp3
167
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/110/110651.mp3
168
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/108/108301.mp3
169
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset

Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/032/032001.mp3
241
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/073/073335.mp3
242
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/081/081814.mp3
243
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/121/121915.mp3
244
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/110/110653.mp3
245
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/092/092874.mp3
246
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/001/001925.mp3
247
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/081/081802.mp3
248
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/127/127066.mp3
249
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset

Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/110/110263.mp3
321
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/023/023155.mp3
322
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/137/137419.mp3
323
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/060/060873.mp3
324
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/057/057822.mp3
325
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/042/042463.mp3
326
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/115/115470.mp3
327
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/111/111385.mp3
328
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/148/148288.mp3
329
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset

...

Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/125/125188.mp3
641
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/071/071693.mp3
642
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/130/130992.mp3
643
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/052/052129.mp3
644
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/121/121595.mp3
645
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/044/044803.mp3
646
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/126/126667.mp3
647
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/125/125432.mp3
648
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/097/097848.mp3
649
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset

Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/110/110204.mp3
721
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/010/010672.mp3
722
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/128/128476.mp3
723
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/099/099375.mp3
724
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/127/127350.mp3
725
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/055/055121.mp3
726
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/011/011262.mp3
727
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/068/068353.mp3
728
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/131/131791.mp3
729
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset

Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/121/121661.mp3
801
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/140/140872.mp3
802
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/065/065755.mp3
803
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/149/149690.mp3
804
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/125/125681.mp3
805
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/109/109687.mp3
806
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/092/092554.mp3
807
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/038/038894.mp3
808
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/131/131938.mp3
809
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset

Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/111/111311.mp3
881
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/129/129910.mp3
882
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/154/154305.mp3
883
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/119/119939.mp3
884
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/042/042129.mp3
885
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/032/032438.mp3
886
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/072/072076.mp3
887
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/064/064860.mp3
888
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/140/140926.mp3
889
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset

Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/017/017906.mp3
961
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/014/014742.mp3
962
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/087/087324.mp3
963
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/097/097211.mp3
964
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/087/087069.mp3
965
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/066/066782.mp3
966
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/057/057418.mp3
967
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/108/108474.mp3
968
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/091/091349.mp3
969
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset

In [103]:
w = len(a)

In [104]:
st = []

for i in range(w):
    for j in range(len(a[i])):
        if len(a[i][j]) != 258:
            st.append(i)

In [105]:
st = list(set(st))
print(len(st))

0


In [106]:
del_len = len(st)
for j in range(del_len):
    ind=del_len-j-1
    a.pop(st[ind])
    b.pop(st[ind])

In [107]:
print(len(a),len(b))

19000 19000


In [108]:
fileXtrain = 'E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/Dataset/ForCNNList/test_setX1'
outfile = open(fileXtrain,'wb')
pickle.dump(a,outfile)
outfile.close()

In [109]:
fileXtrain = 'E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/Dataset/ForCNNList/test_sety1'
outfile = open(fileXtrain,'wb')
pickle.dump(b,outfile)
outfile.close()

# Test - 2

In [110]:
X_train12, y_train12 = split_then_mels(X_test2, y_test2)
a = X_train12
b = y_train12

Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/032/032437.mp3
0
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/108/108049.mp3
1
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/043/043598.mp3
2
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/044/044946.mp3
3
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/091/091900.mp3
4
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/087/087158.mp3
5
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/111/111229.mp3
6
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/023/023015.mp3
7
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/124/124201.mp3
8
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/075/075

Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/037/037911.mp3
81
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/071/071937.mp3
82
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/061/061174.mp3
83
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/073/073573.mp3
84
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/059/059449.mp3
85
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/150/150265.mp3
86
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/111/111146.mp3
87
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/126/126014.mp3
88
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/119/119582.mp3
89
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_smal

Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/128/128500.mp3
161
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/112/112527.mp3
162
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/035/035549.mp3
163
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/123/123001.mp3
164
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/033/033426.mp3
165
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/090/090828.mp3
166
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/006/006606.mp3
167
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/085/085952.mp3
168
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/126/126669.mp3
169
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset

Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/141/141303.mp3
241
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/144/144180.mp3
242
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/068/068896.mp3
243
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/108/108906.mp3
244
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/116/116454.mp3
245
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/129/129884.mp3
246
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/120/120178.mp3
247
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/116/116239.mp3
248
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/145/145431.mp3
249
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset

Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/116/116458.mp3
321
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/056/056031.mp3
322
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/121/121987.mp3
323
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/063/063161.mp3
324
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/126/126557.mp3
325
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/000/000255.mp3
326
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/013/013670.mp3
327
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/063/063471.mp3
328
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/137/137722.mp3
329
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset

...

Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/009/009511.mp3
641
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/126/126778.mp3
642
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/073/073792.mp3
643
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/016/016334.mp3
644
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/027/027177.mp3
645
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/075/075762.mp3
646
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/097/097989.mp3
647
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/028/028375.mp3
648
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/092/092562.mp3
649
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset

Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/037/037111.mp3
721
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/120/120467.mp3
722
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/124/124702.mp3
723
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/041/041570.mp3
724
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/072/072068.mp3
725
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/101/101951.mp3
726
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/107/107179.mp3
727
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/134/134795.mp3
728
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/141/141875.mp3
729
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset

Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/122/122651.mp3
801
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/127/127289.mp3
802
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/025/025324.mp3
803
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/105/105408.mp3
804
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/075/075441.mp3
805
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/061/061011.mp3
806
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/004/004067.mp3
807
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/062/062534.mp3
808
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/051/051006.mp3
809
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset

Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/133/133573.mp3
881
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/064/064789.mp3
882
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/124/124470.mp3
883
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/021/021672.mp3
884
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/074/074389.mp3
885
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/014/014358.mp3
886
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/067/067016.mp3
887
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/142/142551.mp3
888
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/115/115473.mp3
889
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset

Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/069/069791.mp3
961
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/004/004511.mp3
962
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/011/011784.mp3
963
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/133/133273.mp3
964
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/024/024370.mp3
965
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/128/128484.mp3
966
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/108/108525.mp3
967
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/072/072468.mp3
968
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/110/110102.mp3
969
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset

In [111]:
w = len(a)

In [112]:
st = []

for i in range(w):
    for j in range(len(a[i])):
        if len(a[i][j]) != 258:
            st.append(i)

In [113]:
st = list(set(st))
print(len(st))

0


In [114]:
del_len = len(st)
for j in range(del_len):
    ind=del_len-j-1
    a.pop(st[ind])
    b.pop(st[ind])

In [115]:
print(len(a),len(b))

19000 19000


In [116]:
fileXtrain = 'E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/Dataset/ForCNNList/test_setX2'
outfile = open(fileXtrain,'wb')
pickle.dump(a,outfile)
outfile.close()

In [117]:
fileXtrain = 'E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/Dataset/ForCNNList/test_sety2'
outfile = open(fileXtrain,'wb')
pickle.dump(b,outfile)
outfile.close()

# Dataset of 1500 data

In [5]:
import pickle
fileXtrain = 'E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/Dataset/ForCNNList/train_setX2'
infile = open(fileXtrain,'rb')
X = pickle.load(infile)
infile.close()

In [6]:
X = np.array(X)

In [7]:
np.save('E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/Dataset/ForCNNList/dataX1', X) 

In [8]:
fileXtrain = 'E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/Dataset/ForCNNList/train_sety2'
infile = open(fileXtrain,'rb')
y = pickle.load(infile)
infile.close()

In [9]:
y = np.array(y)

In [10]:
np.save('E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/Dataset/ForCNNList/datay1', y) 

In [11]:
fileXtrain = 'E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/Dataset/ForCNNList/train_setX3'
infile = open(fileXtrain,'rb')
X = pickle.load(infile)
infile.close()

In [12]:
X = X[0:9500]

In [13]:
X = np.array(X)

In [14]:
np.save('E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/Dataset/ForCNNList/dataX2', X) 

In [15]:
fileXtrain = 'E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/Dataset/ForCNNList/train_sety3'
infile = open(fileXtrain,'rb')
y = pickle.load(infile)
infile.close()

In [16]:
y = y[0:9500]

In [17]:
y = np.array(y)

In [18]:
np.save('E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/Dataset/ForCNNList/datay2', y) 

In [19]:
del X

In [20]:
del y

In [ ]:
fileXtrain = 'E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/Dataset/ForCNNList/dataX1.npy'
#infile = open(fileXtrain,'rb')
X = np.load(fileXtrain)
#infile.close()

In [ ]:
fileXtrain = 'E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/Dataset/ForCNNList/dataX2.npy'
#infile = open(fileXtrain,'rb')
X2 = np.load(fileXtrain)
#infile.close()

In [ ]:
print(X.shape, X2.shape)

In [ ]:
X = np.concatenate((X,X2))
print(X.shape)

# CNN Model

In [1]:
import os
import h5py
import itertools
from copy import copy
import numpy as np
import matplotlib.pyplot as plt
from collections import OrderedDict
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
import os
import itertools
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import kurtosis
from scipy.stats import skew
import tensorflow as tf
from tensorflow import keras

In [2]:
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Add
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import PReLU
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import AveragePooling2D
from tensorflow.keras.layers import GlobalAveragePooling2D
from tensorflow.keras.layers import GlobalMaxPooling2D
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.callbacks import ReduceLROnPlateau
import pickle
np.random.seed(467)

In [3]:
fileXtrain = 'E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/Dataset/ForCNNList/dataX1.npy'
#infile = open(fileXtrain,'rb')
X = np.load(fileXtrain)
#infile.close()

In [4]:
fileXtrain = 'E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/Dataset/ForCNNList/datay1.npy'
#infile = open(fileXtrain,'rb')
y = np.load(fileXtrain)
#infile.close()

In [5]:
print(X.shape, y.shape)

(19000, 128, 258, 1) (19000,)


In [6]:
X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.15, random_state=47, stratify=y)

In [8]:
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(16150, 128, 258, 1) (2850, 128, 258, 1) (16150,) (2850,)


In [20]:
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

In [23]:
print(y_train.shape, y_test.shape)

(16150, 8) (2850, 8)


In [24]:
genres = {
    'Hip-Hop': 0, 'Pop': 1, 'Folk': 2, 'Experimental': 3, 'Rock': 4, 
    'International': 5, 'Electronic': 6, 'Instrumental': 7
}

# Data Augmentation

In [25]:
from tensorflow.keras.utils import Sequence
class GTZANGenerator(Sequence):
    def __init__(self, X, y, batch_size=64, is_test = False):
        self.X = X
        self.y = y
        self.batch_size = batch_size
        self.is_test = is_test
    
    def __len__(self):
        return int(np.ceil(len(self.X)/self.batch_size))
    
    def __getitem__(self, index):
        # Get batch indexes
        signals = self.X[index*self.batch_size:(index+1)*self.batch_size]

        # Apply data augmentation
        if not self.is_test:
            signals = self.__augment(signals)
        return signals, self.y[index*self.batch_size:(index+1)*self.batch_size]
    
    def __augment(self, signals, hor_flip = 0.5, random_cutout = 0.5):
        spectrograms =  []
        for s in signals:
            signal = copy(s)
            
            # Perform horizontal flip
            if np.random.rand() < hor_flip:
                signal = np.flip(signal, 1)

            # Perform random cutoout of some frequency/time
            if np.random.rand() < random_cutout:
                lines = np.random.randint(signal.shape[0], size=3)
                cols = np.random.randint(signal.shape[0], size=4)
                signal[lines, :, :] = -80 # dB
                signal[:, cols, :] = -80 # dB

            spectrograms.append(signal)
        return np.array(spectrograms)
    
    def on_epoch_end(self):
        self.indexes = np.arange(len(self.X))
        np.random.shuffle(self.indexes)
        return None

# Defining the Network

In [26]:
def conv_block(x, n_filters, pool_size=(2, 2)):
    x = Conv2D(n_filters, (3, 3), strides=(1, 1), padding='same')(x)
    x = Activation('relu')(x)
    x = MaxPooling2D(pool_size=pool_size, strides=pool_size)(x)
    x = Dropout(0.25)(x)
    return x

In [27]:
# Model Definition
def create_model(input_shape, num_genres):
    inpt = Input(shape=input_shape)
    x = conv_block(inpt, 16)
    x = conv_block(x, 32)
    x = conv_block(x, 64)
    x = conv_block(x, 128)
    x = conv_block(x, 256)
    
    # Global Pooling and MLP
    x = Flatten()(x)
    x = Dropout(0.5)(x)
    x = Dense(512, activation='relu', 
              kernel_regularizer=tf.keras.regularizers.l2(0.02))(x)
    x = Dropout(0.25)(x)
    predictions = Dense(num_genres, 
                        activation='softmax', 
                        kernel_regularizer=tf.keras.regularizers.l2(0.02))(x)
    
    model = Model(inputs=inpt, outputs=predictions)
    return model

In [28]:
model = create_model(X_train[0].shape, 8)

In [29]:
model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 128, 258, 1)]     0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 128, 258, 16)      160       
_________________________________________________________________
activation_5 (Activation)    (None, 128, 258, 16)      0         
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 64, 129, 16)       0         
_________________________________________________________________
dropout_7 (Dropout)          (None, 64, 129, 16)       0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 64, 129, 32)       4640      
_________________________________________________________________
activation_6 (Activation)    (None, 64, 129, 32)       0   

In [30]:
model.compile(loss=tf.keras.losses.categorical_crossentropy,
              optimizer=tf.keras.optimizers.Adam(),
              metrics=['accuracy'])

In [31]:
reduceLROnPlat = ReduceLROnPlateau(
    monitor='val_loss', 
    factor=0.95,
    patience=3,
    verbose=1,
    mode='min',
    min_delta=0.0001,
    cooldown=2,
    min_lr=1e-5
)

In [32]:
# Generators
batch_size = 128
train_generator = GTZANGenerator(X_train, y_train)
steps_per_epoch = np.ceil(len(X_train)/batch_size)

validation_generator = GTZANGenerator(X_test, y_test)
val_steps = np.ceil(len(X_test)/batch_size)

In [33]:
import tensorflow as tf
print("Num GPUs Available", len(tf.config.experimental.list_physical_devices('GPU')))

Num GPUs Available 1


In [35]:
hist = model.fit_generator(
    train_generator,
    steps_per_epoch=steps_per_epoch,
    validation_data=validation_generator,
    validation_steps=val_steps,
    epochs=200,
    verbose=1,
    callbacks=[reduceLROnPlat])

Epoch 1/200
127/127 [==============================] - 320s 3s/step - loss: 14.4820 - accuracy: 0.1869 - val_loss: 8.1822 - val_accuracy: 0.2188
Epoch 2/200
127/127 [==============================] - 160s 1s/step - loss: 5.9425 - accuracy: 0.2229 - val_loss: 4.4240 - val_accuracy: 0.2317
Epoch 3/200
127/127 [==============================] - 131s 1s/step - loss: 3.6824 - accuracy: 0.2519 - val_loss: 3.1323 - val_accuracy: 0.2629
Epoch 4/200
127/127 [==============================] - 104s 823ms/step - loss: 2.8120 - accuracy: 0.2594 - val_loss: 2.6028 - val_accuracy: 0.2731
Epoch 5/200
127/127 [==============================] - 88s 695ms/step - loss: 2.4335 - accuracy: 0.2734 - val_loss: 2.2812 - val_accuracy: 0.3145
Epoch 6/200
127/127 [==============================] - 74s 584ms/step - loss: 2.2446 - accuracy: 0.2948 - val_loss: 2.1673 - val_accuracy: 0.2982
Epoch 7/200
127/127 [==============================] - 74s 585ms/step - loss: 2.1533 - accuracy: 0.3044 - val_loss: 2.0739 - val

127/127 [==============================] - 72s 570ms/step - loss: 1.4795 - accuracy: 0.5832 - val_loss: 1.3829 - val_accuracy: 0.6155
Epoch 55/200
127/127 [==============================] - 72s 568ms/step - loss: 1.4476 - accuracy: 0.5911 - val_loss: 1.4029 - val_accuracy: 0.5971
Epoch 56/200
127/127 [==============================] - 72s 570ms/step - loss: 1.4533 - accuracy: 0.5892 - val_loss: 1.3476 - val_accuracy: 0.6223
Epoch 57/200
127/127 [==============================] - 72s 568ms/step - loss: 1.4455 - accuracy: 0.5977 - val_loss: 1.3527 - val_accuracy: 0.6155
Epoch 58/200
127/127 [==============================] - 72s 571ms/step - loss: 1.4251 - accuracy: 0.6004 - val_loss: 1.3163 - val_accuracy: 0.6481
Epoch 59/200
127/127 [==============================] - 72s 568ms/step - loss: 1.4252 - accuracy: 0.5982 - val_loss: 1.3234 - val_accuracy: 0.6291
Epoch 60/200
127/127 [==============================] - 72s 570ms/step - loss: 1.4140 - accuracy: 0.6006 - val_loss: 1.3929 - val_a

127/127 [==============================] - 72s 571ms/step - loss: 1.0926 - accuracy: 0.7217 - val_loss: 0.9538 - val_accuracy: 0.7765
Epoch 108/200
127/127 [==============================] - 72s 571ms/step - loss: 1.0704 - accuracy: 0.7199 - val_loss: 0.9554 - val_accuracy: 0.7819
Epoch 109/200
127/127 [==============================] - 73s 571ms/step - loss: 1.0765 - accuracy: 0.7215 - val_loss: 0.9322 - val_accuracy: 0.7962
Epoch 110/200
127/127 [==============================] - 72s 568ms/step - loss: 1.0739 - accuracy: 0.7252 - val_loss: 0.9548 - val_accuracy: 0.7751
Epoch 111/200
127/127 [==============================] - 72s 570ms/step - loss: 1.0676 - accuracy: 0.7251 - val_loss: 0.9337 - val_accuracy: 0.7996
Epoch 112/200
127/127 [==============================] - 72s 568ms/step - loss: 1.0422 - accuracy: 0.7356 - val_loss: 0.9479 - val_accuracy: 0.7799

Epoch 00112: ReduceLROnPlateau reducing learning rate to 0.0005688000208465382.
Epoch 113/200
127/127 [======================

127/127 [==============================] - 73s 571ms/step - loss: 0.8299 - accuracy: 0.8011 - val_loss: 0.7119 - val_accuracy: 0.8438
Epoch 160/200
127/127 [==============================] - 73s 571ms/step - loss: 0.8166 - accuracy: 0.7953 - val_loss: 0.7075 - val_accuracy: 0.8580
Epoch 161/200
127/127 [==============================] - 72s 568ms/step - loss: 0.8437 - accuracy: 0.7875 - val_loss: 0.7252 - val_accuracy: 0.8451

Epoch 00161: ReduceLROnPlateau reducing learning rate to 0.00041812024719547477.
Epoch 162/200
127/127 [==============================] - 73s 571ms/step - loss: 0.8111 - accuracy: 0.8014 - val_loss: 0.6971 - val_accuracy: 0.8471
Epoch 163/200
127/127 [==============================] - 72s 569ms/step - loss: 0.8015 - accuracy: 0.8021 - val_loss: 0.7214 - val_accuracy: 0.8410
Epoch 164/200
127/127 [==============================] - 72s 571ms/step - loss: 0.8200 - accuracy: 0.7932 - val_loss: 0.7112 - val_accuracy: 0.8397
Epoch 165/200
127/127 [=====================

In [36]:
model.save("cnn_final3.h5")

In [37]:
score = model.evaluate(X_test, y_test, verbose=0)
print("val_loss = {:.3f} and val_acc = {:.3f}".format(score[0], score[1]))

val_loss = 0.545 and val_acc = 0.888


In [38]:
new_model = tf.keras.models.load_model('cnn_final3.h5')
new_model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 128, 258, 1)]     0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 128, 258, 16)      160       
_________________________________________________________________
activation_5 (Activation)    (None, 128, 258, 16)      0         
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 64, 129, 16)       0         
_________________________________________________________________
dropout_7 (Dropout)          (None, 64, 129, 16)       0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 64, 129, 32)       4640      
_________________________________________________________________
activation_6 (Activation)    (None, 64, 129, 32)       0   